<h1>SASSIE Under Ice Float Data Use and Visualization</h1>

<p> The UIF is a drifting platforms that operate similarly to Argo Float under the sea ice. UIF data are downloaded and plotted using the code below! <b>Please run the 'Data Download and Metadata Viewing' and 'Supporting Code' sections in order before running the 'Figure Making Code'.</b></p>

<p>The aim of this notebook is to assist the end user in exploratory data analysis by downloading the SASSIE data from NASA's PODAAC, opening the dataset and displaying it's associated metadata, and creating a few visualizations. This notebook was created by Elizabeth Westbrook. For questions and trouble shooting, please email westbrooke@uncw.edu.</p>

# Data Download and Metadata Viewing

The code in this section will download the dataset from PO.DAAC and open it as an xarray object for metadata and variable attribute viewing.

In [ ]:
import numpy as np
import xarray as xr
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
import requests 

<h2>Download The UIF data from PO.DAAC</h2>
<p> All data from the SASSIE campaign is stored on NASA's PO.DAAC. The code in this section of the notebook will download SASSIE ALTO/ALAMO Float data from PO.DAAC, which is accessed through earthdata. If you do not already have an earthdata account, you can create one <a href="https://urs.earthdata.nasa.gov/">here</a> . </p>
    <p>Please enter your earthdata credentials below.</p>

In [ ]:
username = 'your_username'
password = 'your_password'
earthdata = requests.auth.HTTPBasicAuth(username, password)

##LOCAL DIRECTORY TO SAVE FLOAT DATA
dir_in = 'Data/UIF/'

The next block of code creates the directory specified above and downloads the Under Ice Float file to your binder session if it has not already been downloaded.<b> To download the dataset to your local disk from here, right click on the file you want to download and click 'download'. </b>

In [ ]:
#if the file has not already been downloaded, it is downloaded here.     
if not os.path.isfile(dir_in+'SASSIE_Fall_2022_under_ice_float.nc'):
    os.makedirs(dir_in)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_UNDER_ICE_FLOAT_V1/SASSIE_Fall_2022_under_ice_float.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(dir_in+'SASSIE_Fall_2022_under_ice_float.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + dir_in+'SASSIE_Fall_2022_under_ice_float.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Under Ice Float file is already in binder directory')

<h2> View The Metadata Inside the UIF File</h2>

The netCDF file has global metadata attributes and attributes associated with each variable. This next block will load data and metadata of the netCDF file into an xarray object (ds). <br> The data set will then be displayed in a clickable HTML format. 


In [ ]:
#See information about the entire dataset:
file = dir_in+'SASSIE_Fall_2022_under_ice_float.nc'
ds = xr.open_dataset(file)
ds

# Supporting Code 
The code in this section provides a set up for the figure making code below by defining directories for data and figures and creating functions that will be called to actually map the data

<h3>Create a Directory to Save Figures</h3>

In [ ]:
#LOCAL DIRECTORY TO SAVE FIGURES
fig_dir ='Figures/UIF/'
#FIGURE DIR 
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

<h3>Define a Colormap and Label for Each Variable in the File</h3>

Within SASSIE's collection of jupyter notebooks, the colormaps used for each variable are held as consistant as possible across all datasets. This function defines the colormap and a label for the variable of interest. 

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET###
def define_variable_attributes(var):
    if (var =='salinity')|(var=='salinity_HRCTD'):
        colormap = 'viridis'
        var_label = 'Salinity'
    if (var =='temperature')|(var=='temperature_HRCTD'):
        colormap = 'plasma'
        var_label = 'Water Temperature ($^{\circ}$C)'
    return colormap,var_label

<h3>Define a Function to Create a Map of the Study Area</h3>

The following function creates a map of the SASSIE study area, which is defined by minimum and maximum lat/lon values. These ranges can be changed later when the function is called to zoom in/out on the study area.

In [ ]:
def map_study_area(latmin, latmax,lonmin,lonmax):
    
    global fig 
    global ax
    
    #create the map as a figure, set the lat and lon ranges, and add land + river data:
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50',zorder=1)
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')
    #Add lat and lon gridlines and labels:
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) #draw_labels=True gives lat labels.
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,75,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-170,-140,2))
    gl.top_labels = False
    gl.bottom_labels = True 
    gl.right_labels = False
    
    #Add markers for reference cities on the coast:
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)
        

<h3>Define a Function to Create a 3D Grid of Latitude, Longitude, and Depth</h3>

The following function creates a 3D grid for plotting 3 dimensional profile data from the UIF

In [ ]:
def make_3d_grid(latmin,latmax,lonmin,lonmax,dmin,dmax):    
        global fig 
        global ax
        
        fig = plt.figure(figsize=(10,8))
        ax = plt.axes(projection='3d',computed_zorder=False)
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False    


        ax.set_xlim(lonmin, lonmax); ax.set_ylim(latmin,latmax); ax.set_zlim(dmin,dmax);
        ax.get_xaxis().get_major_formatter().set_useOffset(False)
        xticks = list(np.linspace(lonmin,lonmax,4))
        ax.set_xticks(np.around(xticks,2))
        ax.set_xticklabels(np.around(xticks,2),rotation=30, ha='center',va='center', minor=False,size='small')

        yticks = list(np.linspace(latmin,latmax,4))
        ax.set_yticks(np.around(yticks,2))
        ax.invert_yaxis()
        ax.set_yticklabels(np.around(yticks,2),rotation=-15,va='bottom', minor=False,size='small')

        zticks = list(np.linspace(dmin,dmax,4))
        ax.set_zticks(np.around(zticks,2))
        ax.set_zticklabels(np.around(zticks,2),ha='center',va='center', minor=False,size='small')

        ax.set_xlabel('Longitude',fontsize=13,labelpad=10,fontweight='bold')
        ax.set_ylabel('Latitude',fontsize=13,labelpad = 7,fontweight='bold')
        ax.set_zlabel('Pressure (dbar)',fontsize=13,labelpad = 16,fontweight='bold')
        ax.view_init(195,280)
        fig.canvas.draw()
        plt.tight_layout()

<h3>Configure Supporting Data to Add to Maps</h3>

<p>The functions for viewing and plotting this data set below have options to include bathymetr and/or shiptrack data to add context to maps. If you are using these options, run
    <br>the following code blocks to:
    <br>1. Create a directory for SASSIE Ship Track data and acess bathymetry data from NOAA
    <br>2. Define functions that add these data to your map when called.</p>

<h4>Create Directory for Shiptrack Data and Access Bathymetry Data </h4>

In [ ]:
## DIRECTORY TO SHIP TRACK DATA
ship_dir =  'Data/TSG/' 

#DOWLOAD SHIPTRACK DATA
if not os.path.isfile(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc'):
    os.makedirs(ship_dir)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_TSG_V1/SASSIE_Fall_2022_Shipboard_TSG.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard TSG file is already in local directory')
    
    
#READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Index Relevant Bathymetry Data and Add it to the Map</h4>
This function will index bathymetry data from NOAA within the appropriate spatial range and add it to the map. 


In [ ]:
def add_bathy_data(latmin,latmax,lonmin,lonmax):
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)
        
        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)

<h4>Define a Function to Add Shiptrack Data to a Map</h4>
This function will pull the lat/lon data from the SASSIE Shipboard TSG file and put it onto a map. 

In [ ]:
def add_ship_track():
        ds_ship = xr.open_dataset(ship_dir+'/SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        ship_lat = np.squeeze(ds_ship['latitude'])
        ship_lon = np.squeeze(ds_ship['longitude'])
       
        track = ax.plot(ship_lon, 
                     ship_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)

# Figure Making Code

## Make a Map of the Surfacing Locations of the UIF Throughout the Campaign

The map_UIF_GPS function shows the track of the UIF as it surfaced on a map, colored by time. Various features of this function:
    <br>1. The function automatically adds the track of the R/V Woldstad in Black for reference. 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 

In [ ]:
##USE THIS TO PLOT THE FLOAT SURFACING LOCATIONS ON THE WHOLE CAMPAIGN MAP 
def map_UIF_GPS(ship_track=True,bathymetry_data=True):
    
    ##CREATE A MAP WITH LAND AND CITY MARKERS AND SHIP TRACK. To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140.  
    latmin = 72
    latmax =74
    lonmin=-156
    lonmax=-149
    

    #make the map: 
    map_study_area(latmin,latmax,lonmin,lonmax)
    
    ax.set_title('Under Ice Float GPS Readings',fontsize=20,pad=1.5) 
   
   
    ##OPTIONAL MAP ADD-ONs
    if ship_track==True:
        add_ship_track()
        
    if bathymetry_data == True:    
        add_bathy_data(latmin,latmax,lonmin,lonmax)
    
    ##APPLY THE TIME AND LOCATION DATA FROM THE GPS 
    ds_UIF = xr.open_dataset(file)
    GPS_time = np.squeeze(ds_UIF['time_GPS'])
    GPS_lat = np.squeeze(ds_UIF['latitude_GPS'])
    GPS_lon = np.squeeze(ds_UIF['longitude_GPS'])
    
    #Set time limits of the float data
    start_time = min(GPS_time)
    end_time = max(GPS_time)

    var_min = start_time.astype('int64')
    var_max =end_time.astype('int64')

    deployment_track = ax.scatter(GPS_lon,GPS_lat,s = 5,
                   c = GPS_time,cmap = 'jet',
                   transform=cartopy.crs.PlateCarree(),zorder=2,vmin=var_min,vmax=var_max)

    cbar = fig.colorbar(deployment_track, ax=ax, orientation="horizontal", pad=0.1)
    cbar.set_label(label='Date',size='large',weight='bold')
    cbar_tick_array=(np.linspace(start_time.astype('int64'),end_time.astype('int64'),5))
    cbar.set_ticks(cbar_tick_array)
    cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)  

    #SAVE FIGURE
    if not os.path.exists(fig_dir+'maps/'):
        os.makedirs(fig_dir+'maps/')
    print('Saving Output Image:  '+fig_dir+'maps/UIF_GPS_readings.png')
    plt.savefig(fig_dir+'maps/UIF_GPS_readings_.png',dpi='figure',format='png')
    

In [ ]:
map_UIF_GPS()

<h2>Three Dimensional Representation of Vertical Profile Data</h2>
Use the code in this section to plot profile locations and depths of the Under Ice float on a 3D grid, colored by a physical measurement variable from the file (temperature, salinity). 

In [ ]:
##MAKES A 3D PLOT OF THE CTD VARIABLES WITH INTERPOLATED LAT/LON FOR A TIME RANGE WITHIN THE GPS MEASUREMENTS
def plot_CTD(start_time,end_time,CTD_unit,var):
   
    ##PULL OUT THE DATA WITHIN THE START AND END TIME RANGE
    ds_limited = ds.sel(time_GPS=slice(start_time,end_time),time_CTD=slice(start_time,end_time))
    
    ## MAKE AN INTERPOLATED CTD LAT AND LON ARRAY FOR PLOTTING PURPOSES ONLY. PULL OUT THE VARIABLE OF INTEREST AND PRESSURE AT THESE TIMES TOO. 
    lat_array = []
    lon_array = []
    pres_array =[]
    var_array = []
    for i in range(len(ds_limited.time_GPS)-1):
        #STEP 1 Find start and end coordinates 
        start_time_GPS = ds_limited['time_GPS'].values[i]
        end_time_GPS = ds_limited['time_GPS'].values[i+1]
        start_lat = ds_limited['latitude_GPS'].values[i]
        end_lat = ds_limited['latitude_GPS.values'][i+1]
        start_lon = ds_limited['longitude_GPS'].values[i]
        end_lon = ds_limited['longitude_GPS'].values[i+1]
        # STEP 2 Find the number of measurements taken between start_time and end_time 
        measurements = ds_limited.sel(time_CTD=slice(start_time_GPS,end_time_GPS))
        number_of_measurements = len(measurements.time_CTD.values)
        #STEP 3 create an evenly spaced array of lat and lons connecting them. 
        lat_array = np.append(lat_array,np.linspace(start_lat,end_lat,number_of_measurements))
        lon_array = np.append(lon_array,np.linspace(start_lon,end_lon,number_of_measurements))
        #STEP 4 create a pressure and a var array that correspond to these measurements.
        pressure = measurements['pressure'].values[CTD_unit-1,:]
        pres_array = np.append(pres_array,pressure)
        var_current = measurements[var].values[CTD_unit-1,:]
        var_array = np.append(var_array,var_current)
    
    ##SET UP THE 3D PLOT 
    colormap,var_label = define_variable_attributes(var)

    #To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140
    latmin = min(lat_array)
    latmax = max(lat_array)
    lonmin = min(lon_array)
    lonmax = max(lon_array)
    pmin = min(pres_array)
    pmax = max(pres_array)+0.5
    
    
    make_3d_grid(latmin,latmax,lonmin,lonmax,pmin,pmax)
    
    
    ax.set_title('Under Ice Float'+' '+var_label+' '+'Readings from CTD Sensor '+str(CTD_unit)+'\n from '+str(start_time.year)+'/'+str(start_time.month).zfill(2)+'/'+str(start_time.day).zfill(2)+' to '+str(end_time.year)+'/'+str(end_time.month).zfill(2)+'/'+str(end_time.day).zfill(2),fontsize=15,pad=None)

    
    ##APPLY THE FLOAT DATA
    a = ax.scatter(lon_array,lat_array,pres_array,c=(var_array),cmap = colormap,s=2,zorder=1)    
    fig.colorbar(a).set_label(label = var_label,size=15,weight='bold')
    
    
    ##SAVE THE FIGURE:
    if not os.path.exists(fig_dir+var+'/'):
        os.makedirs(fig_dir+var+'/')
    
    print('Saving Output Image:  '+fig_dir+var+'/'+'UIF_CTD'+str(CTD_unit)+'_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png')
    plt.savefig(fig_dir+var+'/'+'UIF_CTD'+str(CTD_unit)+'_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png',dpi='figure',format='png',bbox_inches='tight')
    



In [ ]:
plot_CTD(datetime(2022,10,14),datetime(2022,10,23),1,'temperature')
plot_CTD(datetime(2022,10,14),datetime(2022,10,23),1,'salinity')

In [ ]:
##MAKES A 3D PLOT OF THE HRCTD VARIABLES WITH INTERPOLATED LAT/LON FOR A TIME RANGE WITHIN THE GPS MEASUREMENTS
def plot_HRCTD(start_time,end_time,var,var_min,var_max):
   
    ##PULL OUT THE DATA WITHIN THE START AND END TIME RANGE
    ds_limited = ds.sel(time_GPS=slice(start_time,end_time),time_HRCTD=slice(start_time,end_time))
    
    ## MAKE AN INTERPOLATED CTD LAT AND LON ARRAY FOR PLOTTING PURPOSES ONLY. PULL OUT THE VARIABLE OF INTEREST AND PRESSURE AT THESE TIMES TOO. 
    lat_array = []
    lon_array = []
    pres_array = []
    var_array = []
    for i in range(len(ds_limited.time_GPS)-1):
        #STEP 1 Find start and end coordinates 
        start_time_GPS = ds_limited['time_GPS'].values[i]
        end_time_GPS = ds_limited['time_GPS'].values[i+1]
        start_lat = ds_limited['latitude_GPS'].values[i]
        end_lat = ds_limited['latitude_GPS.values'][i+1]
        start_lon = ds_limited['longitude_GPS'].values[i]
        end_lon = ds_limited['longitude_GPS'].values[i+1]
        # STEP 2 Find the number of measurements taken between start_time and end_time 
        measurements = ds_limited.sel(time_HRCTD=slice(start_time_GPS,end_time_GPS))
        number_of_measurements = len(measurements.time_HRCTD.values)
        #STEP 3 create an evenly spaced array of lat and lons connecting them. 
        lat_array = np.append(lat_array,np.linspace(start_lat,end_lat,number_of_measurements))
        lon_array = np.append(lon_array,np.linspace(start_lon,end_lon,number_of_measurements))
        #STEP 4 create a pressure and a var array that correspond to these measurements.
        pressure = measurements['pressure_HRCTD'].values
        pres_array = np.append(pres_array,pressure)
        var_current = measurements[var+'_HRCTD'].values
        var_array = np.append(var_array,var_current)
    
    
    ##SET UP THE 3D PLOT 
    colormap,var_label = define_variable_attributes(var)

    #To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140
    latmin = min(lat_array)
    latmax = max(lat_array)
    lonmin = min(lon_array)
    lonmax = max(lon_array)
    pmin = min(pres_array)
    pmax = max(pres_array)+0.5

    
    make_3d_grid(latmin,latmax,lonmin,lonmax,pmin,pmax)
    
    ax.set_title('Under Ice Float'+' '+var_label+' '+'Readings from HRCTD' +'\n from '+str(start_time.year)+'/'+str(start_time.month).zfill(2)+'/'+str(start_time.day).zfill(2)+' to '+str(end_time.year)+'/'+str(end_time.month).zfill(2)+'/'+str(end_time.day).zfill(2),fontsize=15,pad=None)

    ##APPLY THE FLOAT DATA
    a = ax.scatter(lon_array,lat_array,pres_array,c=(var_array),cmap = colormap,s=2,zorder=1,vmin = var_min, vmax = var_max)    
    fig.colorbar(a).set_label(label = var_label,size = 15,weight='bold')
    
    ##SAVE THE FIGURE:
    if not os.path.exists(fig_dir+'HRCTD/'+var+'/'):
        os.makedirs(fig_dir+'HRCTD/'+var+'/')
    
    print('Saving Output Image:  '+fig_dir+'HRCTD/'+var+'/'+'UIF_HRCTD_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png')
    plt.savefig(fig_dir+'HRCTD/'+var+'/'+'UIF_HRCTD_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png',dpi='figure',format='png',bbox_inches='tight')
    



In [ ]:
plot_HRCTD(datetime(2022,10,14),datetime(2022,10,23),'temperature',-1.5,0)
plot_HRCTD(datetime(2022,10,14),datetime(2022,10,23),'salinity',25,29)

<h2>Two Dimensional Representation of Vertical Profile Data</h2>
Use the code in this section to plot profile locations and depths of the Under Ice float on a 2D grid, colored by a physical measurement variable from the file (temperature, salinity). 

In [ ]:
##MAKES A 2D PLOT OF THE CTD VARIABLES ON TIME-DEPTH AXIS (NO INTERPOLATION OF LAT/LON)
def plot_CTD_2D(start_time,end_time,CTD_unit,var):
   
    ##PULL OUT THE DATA WITHIN THE START AND END TIME RANGE
    ds_limited = ds.sel(time_CTD=slice(start_time,end_time))
    UIF_time = ds_limited['time_CTD'].values
    UIF_pres = ds_limited['pressure'].values[CTD_unit-1,:]
    UIF_var = ds_limited[var].values[CTD_unit-1,:]            
    
    ##SET UP THE 2D PLOT 
    colormap,var_label = define_variable_attributes(var)

    pmin = min(UIF_pres)
    pmax = max(UIF_pres)
    tmin = min(UIF_time)
    tmax = max(UIF_time)
                  
    fig = plt.figure(figsize=(10,8))
    
    plt.title('Under Ice Float'+' '+var_label+' '+'Readings from CTD Sensor '+str(CTD_unit)+'\n from '+str(start_time.year)+'/'+str(start_time.month).zfill(2)+'/'+str(start_time.day).zfill(2)+' to '+str(end_time.year)+'/'+str(end_time.month).zfill(2)+'/'+str(end_time.day).zfill(2),fontsize=15,pad=None)

    plt.xlim(tmin, tmax); plt.ylim(pmin,pmax);
    total_timedelta = tmax-tmin
    third_of_timedelta = total_timedelta/3
    plt.xticks([tmin,tmin+third_of_timedelta, tmin+ 2*third_of_timedelta, tmax])
    ytick = list(np.arange(0,75,15))
    plt.yticks(np.around(ytick,2))
    plt.xlabel('Date',fontsize=16)
    plt.ylabel('Pressure (dbar)',fontsize=16)
    plt.gca().invert_yaxis()
        
    ##APPLY THE CTD DATA
    ax = plt.scatter(UIF_time,UIF_pres,s=1,c=UIF_var,cmap=colormap)
    fig.colorbar(ax).set_label(label = var_label,size=15,weight='bold')

    ##SAVE THE FIGURE:
    if not os.path.exists(fig_dir+var+'/2d/'):
        os.makedirs(fig_dir+var+'/2d/')
    
    print('Saving Output Image:  '+fig_dir+var+'/2d/'+'UIF_CTD'+str(CTD_unit)+'_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png')
    plt.savefig(fig_dir+var+'/2d/'+'UIF_CTD'+str(CTD_unit)+'_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png',dpi='figure',format='png',bbox_inches='tight')
    



In [ ]:
plot_CTD_2D(datetime(2022,10,14),datetime(2022,10,23),1,'temperature')
plot_CTD_2D(datetime(2022,10,14),datetime(2022,10,23),1,'salinity')

In [ ]:
##MAKES A 2D PLOT OF THE CTD VARIABLES ON TIME-DEPTH AXIS (NO INTERPOLATION OF LAT/LON)
def plot_HRCTD_2D(start_time,end_time,var,var_min,var_max):
   
    ##PULL OUT THE DATA WITHIN THE START AND END TIME RANGE
    ds_limited = ds.sel(time_HRCTD=slice(start_time,end_time))
    UIF_time = ds_limited['time_HRCTD'].values
    UIF_pres = ds_limited['pressure_HRCTD'].values
    UIF_var = ds_limited[var+'_HRCTD'].values            
    
    ##SET UP THE 2D PLOT 
    colormap,var_label = define_variable_attributes(var)

    pmin = min(UIF_pres)
    pmax = max(UIF_pres)
    tmin = min(UIF_time)
    tmax = max(UIF_time)
                  
    fig = plt.figure(figsize=(10,8))
    
    plt.title('Under Ice Float'+' '+var_label+' '+'Readings from HRCTD' +'\n from '+str(start_time.year)+'/'+str(start_time.month).zfill(2)+'/'+str(start_time.day).zfill(2)+' to '+str(end_time.year)+'/'+str(end_time.month).zfill(2)+'/'+str(end_time.day).zfill(2),fontsize=15,pad=None)

    plt.xlim(tmin, tmax); plt.ylim(pmin,pmax);
    total_timedelta = tmax-tmin
    third_of_timedelta = total_timedelta/3
    plt.xticks([tmin,tmin+third_of_timedelta, tmin+ 2*third_of_timedelta, tmax])
    ytick = list(np.arange(0,40,10))
    plt.yticks(np.around(ytick,2))
    plt.xlabel('Date',fontsize=16)
    plt.ylabel('Pressure (dbar)',fontsize=16)
    plt.gca().invert_yaxis()
        
    ##APPLY THE HRCTD DATA
    ax = plt.scatter(UIF_time,UIF_pres,s=1,c=UIF_var,cmap=colormap,vmin = var_min, vmax = var_max)
    fig.colorbar(ax).set_label(label = var_label,size=15,weight='bold')
    
    ##SAVE THE FIGURE:
    if not os.path.exists(fig_dir+'HRCTD/'+var+'/2d/'):
        os.makedirs(fig_dir+'HRCTD/'+var+'/2d/')
    
    print('Saving Output Image:  '+fig_dir+'HRCTD/'+var+'/2d/'+'UIF_HRCTD_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png')
    plt.savefig(fig_dir+'HRCTD/'+var+'/2d/'+'UIF_HRCTD_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png',dpi='figure',format='png',bbox_inches='tight')
    



In [ ]:
plot_HRCTD_2D(datetime(2022,10,14),datetime(2022,10,23),'temperature',-1.5,0)
plot_HRCTD_2D(datetime(2022,10,14),datetime(2022,10,23),'salinity',25,29)